In [19]:
import pandas as pd

In [ ]:
exp_matrix = pandas.read_csv('expression_matrix_simulated.csv', index_col=0)


In [9]:
class studyset:
    def __init__(self, expression_data):
        self.expression_data = expression_data
        self.control, self.affected = self.identify_control_and_affected()
        self.control_df = self.expression_data[self.control]
        self.affected_df = self.expression_data[self.affected]
    def identify_control_and_affected(self):
        patient_names = self.expression_data.columns.tolist()
        control = [name for name in patient_names if name.startswith('CTRL')]
        affected = [name for name in patient_names if name.startswith('AFF')]
        return control, affected

In [10]:
whodunitcase = studyset(exp_matrix)

In [12]:
whodunitcase.control_df

,CTRL_1,CTRL_2,CTRL_3,CTRL_4,CTRL_5,CTRL_6,CTRL_7,CTRL_8,CTRL_9,CTRL_10,CTRL_11,CTRL_12
gene,,,,,,,,,,,,
ACTB,5.695490,5.869985,5.594361,5.953764,6.143176,5.843125,5.911840,5.807330,5.788755,5.536180,5.899498,5.754675
BCL2A1,6.191783,6.566791,5.894102,6.301209,6.146769,6.193718,6.592847,6.453454,6.693239,6.245668,6.083237,6.224862
CCND1,6.719173,6.495798,6.679095,7.032897,6.262609,6.911132,6.852125,6.602560,6.346987,6.802444,6.621957,6.850608
CDK2,7.311131,7.107872,7.439972,7.034215,6.611904,6.568797,7.474523,7.496344,6.925389,6.864289,7.417678,6.647131
GAPDH,5.543909,5.021309,5.715927,5.401015,5.587556,5.465959,5.739868,5.157655,5.259394,6.518096,5.322262,5.388985
...,...,...,...,...,...,...,...,...,...,...,...,...
MMP9,7.112002,6.667392,6.461697,6.527646,6.599613,7.455722,6.570765,7.068727,6.546302,7.096653,6.932666,6.770189
MYC,10.079919,10.885615,10.971166,9.669946,9.961931,10.605440,10.405603,10.540352,10.164042,10.943641,10.897918,10.577626
MYCN,7.110837,7.132117,7.140257,7.910128,6.763500,6.715878,6.580189,7.223578,7.454425,6.895111,6.568317,6.575371


In [22]:
import scipy
import numpy as np
class statistics:
    def compute_basic_stats(self, studyset):
        control_mean = studyset.control_df.mean(axis=1)
        affected_mean = studyset.affected_df.mean(axis=1)
        control_std = studyset.control_df.std(axis=1)
        affected_std = studyset.affected_df.std(axis=1)
        stats_df = pd.DataFrame({
            'Control_Mean': control_mean,
            'Affected_Mean': affected_mean,
            'Control_Std': control_std,
            'Affected_Std': affected_std
        })
        return stats_df
    def compute_differential_expression(self, studyset):
        stats_df = self.compute_basic_stats(studyset)
        stats_df['Fold_Change'] = stats_df['Affected_Mean'] / stats_df['Control_Mean']
        stats_df['Log2_Fold_Change'] = np.log2(stats_df['Fold_Change'])
        p_values = scipy.stats.ttest_ind(studyset.affected_df.T, studyset.control_df.T, axis=0, equal_var=False).pvalue
        stats_df['P_Value'] = p_values
        stats_df['Adjusted_P_Value'] = scipy.stats.false_discovery_control(stats_df['P_Value'], method='bh')
        return stats_df

In [25]:
#statistics().compute_basic_stats(whodunitcase)
stats_df = statistics().compute_differential_expression(whodunitcase)

In [ ]:
relevant_genes = stats_df[stats_df['P_Value'] < 0.05]
relevant_genes_adj = stats_df[stats_df['Adjusted_P_Value'] < 0.05]

In [30]:
relevant_genes_adj.sort_values(by='Adjusted_P_Value')

,Control_Mean,Affected_Mean,Control_Std,Affected_Std,Fold_Change,Log2_Fold_Change,P_Value,Adjusted_P_Value
gene,,,,,,,,
CCND1,6.681449,30.377178,0.229942,0.281195,4.546496,2.184755,2.727723e-37,2.564060e-35
IL6,7.528055,35.513764,0.447875,0.208927,4.717522,2.238029,7.622426e-28,3.582540e-26
MYC,10.475267,21.123865,0.427895,0.338124,2.016547,1.011887,5.985758e-26,1.875537e-24
MMP9,6.817448,1.407617,0.311251,0.260897,0.206473,-2.275977,7.105082e-23,1.669694e-21
CDK2,7.074937,1.326024,0.352503,0.266728,0.187426,-2.415611,5.665763e-22,1.065163e-20
MLX,12.239971,18.701424,0.261802,0.379452,1.527898,0.611548,7.637070e-22,1.196474e-20
BCL2A1,6.298973,1.195402,0.233926,0.344120,0.189777,-2.397621,1.350751e-20,1.813865e-19
MAX,11.623600,16.244500,0.395870,0.336704,1.397545,0.482894,3.038814e-19,3.570606e-18
MYCN,7.005809,13.879627,0.407251,0.185750,1.981160,0.986345,7.269613e-19,7.592707e-18
